In [277]:
import pandas as pd

In [278]:
data = pd.read_csv('train.csv')

In [279]:
data.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [280]:
data.isnull().sum()

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [281]:
data = data.drop(columns=["id", "CustomerId", "Surname"])

In [282]:
x = data[data.columns[:-1]]
y = data[data.columns[-1]]

In [283]:
# bins = [18, 30, 40, 50, 60, 70, 92]
# labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70+']

# x['AgeCategory'] = pd.cut(x['Age'], bins=bins, labels=labels, include_lowest=True, right=False)


In [284]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
encoder = OneHotEncoder()
categorical_columns = ["Geography"]

for column in categorical_columns:
    categorical = pd.DataFrame(x[column])
    transformed = encoder.fit_transform(categorical)
    transformed_df = pd.DataFrame(transformed.toarray(), columns=encoder.get_feature_names_out([column]))
    x = pd.concat([x, transformed_df], axis=1)
    x.drop(columns=[column], inplace=True)

In [285]:
le = LabelEncoder()
x["Gender"] = le.fit_transform(x["Gender"])

In [286]:
x.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,668,1,33.0,3,0.00,2,1.0,0.0,181449.97,1.0,0.0,0.0
1,627,1,33.0,1,0.00,2,1.0,1.0,49503.50,1.0,0.0,0.0
2,678,1,40.0,10,0.00,2,1.0,0.0,184866.69,1.0,0.0,0.0
3,581,1,34.0,2,148882.54,1,1.0,1.0,84560.88,1.0,0.0,0.0
4,716,1,33.0,5,0.00,2,1.0,1.0,15068.83,0.0,0.0,1.0


In [287]:
y.value_counts()

Exited
0    130113
1     34921
Name: count, dtype: int64

Handling Data Imbalance

In [288]:
from imblearn.over_sampling import RandomOverSampler

randomOverSampler = RandomOverSampler()

x_resampled, y_resampled = randomOverSampler.fit_resample(x,y)

In [289]:
y_resampled.value_counts()

Exited
0    130113
1    130113
Name: count, dtype: int64

In [290]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_resampled)

In [292]:
x_scaled

array([[ 0.16195369,  0.9756411 , -0.77541642, ...,  0.95344685,
        -0.60959245, -0.50819989],
       [-0.34712391,  0.9756411 , -0.77541642, ...,  0.95344685,
        -0.60959245, -0.50819989],
       [ 0.28611896,  0.9756411 , -0.02688739, ...,  0.95344685,
        -0.60959245, -0.50819989],
       ...,
       [ 0.74553045, -1.02496707,  0.93550707, ...,  0.95344685,
        -0.60959245, -0.50819989],
       [-0.08637685, -1.02496707,  0.40084348, ..., -1.04882616,
         1.64044025, -0.50819989],
       [ 1.24219153, -1.02496707, -0.24075283, ..., -1.04882616,
         1.64044025, -0.50819989]])

In [275]:
from sklearn.model_selection import train_test_split
x_latih, x_uji, y_latih, y_uji = train_test_split(x_scaled,y_resampled ,test_size=0.2, random_state=2)

In [276]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV 

# param_grid = {'C': [0.1, 1, 10, 100, 1000],  
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#               'kernel': ['rbf']}  

svm = SVC()
# grid = GridSearchCV(svm, param_grid, refit = True, verbose = 3) 
svm.fit(x_latih, y_latih)

y_pred = svm.predict(x_uji)
akurasi = accuracy_score(y_uji, y_pred)
print("Akurasi Model SVM pada Data Uji:", akurasi)


Akurasi Model SVM pada Data Uji: 0.8010413864658187


In [251]:
data_test = pd.read_csv('test.csv')

In [252]:
id = data_test["id"]
data_test = data_test.drop(columns=['id','CustomerId', 'Surname'])

In [253]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
categorical_columns = ["Geography", "Gender"]

for column in categorical_columns:
    categorical = pd.DataFrame(data_test[column])
    transformed = encoder.fit_transform(categorical)
    transformed_df = pd.DataFrame(transformed.toarray(), columns=encoder.get_feature_names_out([column]))
    data_test = pd.concat([data_test, transformed_df], axis=1)
    data_test.drop(columns=[column], inplace=True)

In [254]:
data_scaled = scaler.fit_transform(data_test)

In [255]:
hasil = svm.predict(data_scaled)

In [256]:
hasil = pd.Series(hasil)

In [257]:
hasil = pd.concat([id, hasil], axis=1)

In [258]:
hasil.head()

,id,0
0,165034,0
1,165035,1
2,165036,0
3,165037,1
4,165038,1


In [259]:
hasil.rename(columns = {0:'Exited'}, inplace=True)

In [260]:
hasil.to_csv('submission_svm.csv', index=False)